In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [72]:
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
# TRAVILY_PROJECT = os.getenv("TRAVILY_PROJECT")

In [73]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")




In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [ ]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hi there! How can I help you today?
goodbye take care yourself


Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Read timed out. (read timeout=10.0)"))'))
Content-Length: 649
API Key: lsv2_********************************************9ctrace=eded666e-ceca-4f9f-99f9-3588d776a494,id=eded666e-ceca-4f9f-99f9-3588d776a494


In [20]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory 
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.history import RunnableWithMessageHistory 


In [21]:
store= {}

In [22]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [23]:

config = {"configurable": {"session_id": "firstchat"}}

In [24]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [25]:

model_with_memory.invoke(("Hi! I'm gagan"),config=config).content

'Hi Gagan! Nice to meet you. How can I help you today?'

In [26]:

model_with_memory.invoke(("Hi! who am i?"),config=config).content

"I have no idea who you are beyond your name, Gagan.  I'm an AI and have no access to real-world information about individuals unless you tell me.  I only know what you've typed in our current conversation.  Is there anything else I can help you with?"

In [28]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
loader = DirectoryLoader('./data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]


db = Chroma.from_documents(new_docs[:10], embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [29]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [30]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [31]:
question="how gagan described himself?"
print(retrieval_chain.invoke(question, config={"configurable": {"session_id": "firstchat"}}))

Gagan Kumar describes himself as "the best boy".


this wwas all about chaat bt an rags before you moove towards agent

TOOLS AND Agents

In [32]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [34]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [35]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [36]:

tool.name

'wikipedia'

In [37]:

tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [38]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


youtube search tool


In [47]:
from langchain_community.tools import YouTubeSearchTool

In [48]:
tool2 = YouTubeSearchTool(api_key=GOOGLE_API_KEY, top_k_results=5)

In [49]:
tool2.name

'youtube_search'

In [50]:
tool2.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [52]:
tool2.run("babloo mehato")

"['https://www.youtube.com/watch?v=HzoVi5YehD8&pp=ygUNYmFibG9vIG1laGF0bw%3D%3D', 'https://www.youtube.com/watch?v=j0lJ0keLJS0&pp=ygUNYmFibG9vIG1laGF0bw%3D%3D']"

In [54]:

from langchain_community.tools.tavily_search import TavilySearchResults

In [55]:
tool4 = TavilySearchResults()


C:\Users\gagan\AppData\Local\Temp\ipykernel_43160\862852305.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool4 = TavilySearchResults()


In [57]:
tool4.invoke({"query": "What happened in the latest burning man floods"})

[{'title': "Satellite Photos Reveal Extent of Floods at This Year's Burning Man",
  'url': 'https://gizmodo.com/satellite-photos-extent-floods-burning-man-1850819889',
  'content': 'The equivalent of several months of rain fell over the area, and the water pooled into the playa where the event takes place every year.',
  'score': 0.98539},
 {'title': 'Burning Man flooding: What happened to stranded festivalgoers?',
  'url': 'https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331',
  'content': 'Tens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.',
  'score': 0.98532},
 {'title': 'After flooding, Burning Man to go on as planned in 2024 - KNPR',
  'url': 'https://knpr.org/news-88-9-knpr/2023-12-27/after-flooding-burning-man-to-go-on-as-planned-in-2024',
  'content': "Burning Man organizers don't foresee major changes in 2024 thanks to a hard-won 

In [58]:

from langchain import hub

In [59]:

from langchain.agents import AgentExecutor,create_openai_functions_agent

creating custom tool

In [61]:
from langchain_core.tools import tool
@tool
def get_word_length(text: str) -> int:
    '''return the length of the text'''
    return len(text)

In [63]:
get_word_length.invoke("ksf")

3

In [64]:
@tool
def multiply(a : int ,b :int) -> int:
    '''return the product of two numbers'''
    return a * b

In [66]:
multiply.invoke({"a":2,"b":3})

6

AGENTS

we are going through the first verson og langcain firtsdt


In [67]:

from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [68]:
tool=load_tools(["wikipedia"],llm=llm)

In [69]:

agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\gagan\AppData\Local\Temp\ipykernel_43160\1791468372.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [70]:

agent.run("what is llama and who create this llm model?")

C:\Users\gagan\AppData\Local\Temp\ipykernel_43160\3196217391.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("what is llama and who create this llm model?")




> Entering new AgentExecutor chain...
Thought: I need to find information about the LLaMA language model.  Specifically, I need to know what it is and who created it.  Wikipedia might be a good source for this.

Action: wikipedia
Action Input: LLaMA
Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 4, released in April 2025.
Llama models come in different sizes, ranging from 1 billion to 2 trillion parameters. Initially only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under l

'LLaMA is a family of large language models created by Meta AI.'

In [75]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()



In [76]:
search.invoke("what is the weather in allahabad")

[{'title': 'Allahabad Weather Today (Saturday, Aug 09, 2025)',
  'url': 'https://www.indiatoday.in/weather/allahabad-weather-forecast-today',
  'content': 'The minimum temperature in Allahabad today is likely to hover around 26 degrees Celsius, while the maximum temperature might reach 32 degrees Celsius. The mercury level is expected to hover around 30 degrees Celsius throughout the day, with the wind speed around 3.94. The wind will move around 226 degrees with a gust speed of 5.32. The sunrise time is 05:33 AM, while it will set at 06:43 PM on Saturday. As per the seven-day weather prediction, the temperature in Allahabad is likely to reach 32 [...] ### TRENDING TOPICS\n\n### Download App\n\n# Allahabad Weather Today (Saturday, Aug 09, 2025)\n\nWeather Icon\nmin\nmax\nrise\nset\nhumdity\npressure\nlon\nlat\nspeed\ndeg\ngust\nRain\nRain\nRain\nRain\nRain\nRain\nRain\n\n## Weather In Allahabad [...] degrees Celsius on Saturday, 35 degrees Celsius on Sunday, 35 degrees Celsius on Monda

In [77]:
tools = [search]

In [78]:

from langchain import hub

In [79]:


prompt=hub.pull("hwchase17/openai-functions-agent")

In [81]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [80]:

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]


agent_scratchpad: The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained. Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field.

In [82]:

from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [83]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [84]:
agent_executor.invoke({"input": "hello how are you?"})




> Entering new AgentExecutor chain...
I'm doing well, thank you for asking. How can I help you today?


> Finished chain.


{'input': 'hello how are you?',
 'output': "I'm doing well, thank you for asking. How can I help you today?\n"}

In [86]:
agent_executor.invoke({"input": "hello can you tell me about the last test cricket match between India and England in 2025?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'last test cricket match between India and England in 2025'}`


[{'title': 'Indian cricket team in England in 2025 - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Indian_cricket_team_in_England_in_2025', 'content': '### 3rd Test\n\n[edit]\n\n10–14 July 2025 \n\nScorecard\n\nEnglandImage 21: EnglandvImage 22: IndiaIndia\n387 (112.3 overs)\n\nJoe Root 104 (199)\n\nJasprit Bumrah 5/74 (27 overs)387 (119.2 overs)\n\nKL Rahul 100 (177)\n\nChris Woakes 3/84 (27 overs)\n192 (62.1 overs)\n\nJoe Root 40 (96)\n\nWashington Sundar 4/22 (12.1 overs)170 (74.5 overs)\n\nRavindra Jadeja 61 (181)\n\nBen Stokes 3/48 (24 overs)\n\nEngland won by 22 runs\n\nLord\'s, London\n\nUmpires: Paul Reiffel (Aus) and Sharfuddoula (Ban) [...] ### 2nd Test\n\n[edit]\n\n2–6 July 2025 \n\nScorecard\n\nIndiaImage 19: IndiavImage 20: EnglandEngland\n587 (151 overs)\n\nShubman Gill 269 (387)\n\nShoaib Bashir 3/167 (45 over

{'input': 'hello can you tell me about the last test cricket match between India and England in 2025?',
 'output': 'The last test match between India and England in 2025 was the 5th test of the series, played at The Oval in London from July 31st to August 4th. India won the match by 6 runs.  India scored 224 and 396, while England scored 247 and 367.\n'}